In [11]:
import numpy as np
import pandas as pd
from nltk.tokenize.stanford import StanfordTokenizer
import nltk
import spacy
sp = spacy.load('en_core_web_sm')

In [2]:
# Installs Spacy
# import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en

     |████████████████████████████████| 10.2 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 968 kB 260 kB/s eta 0:00:01
     |████████████████████████████████| 262 kB 688 kB/s eta 0:00:01
     |████████████████████████████████| 286 kB 890 kB/s eta 0:00:01
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
     |████████████████████████████████| 5.8 MB 499 kB/s eta 0:00:01
  Using cached wasabi-0.8.0-py3-none-any.whl (23 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
     |████████████████████████████████| 12.0 MB 493 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=1db14003c4095e2547a6a077a64569181af2ab4a565f97831c9a84c1e565390f
  Stored in directory: /private/var/folders/69/lwzwt6750571139n1c3vryr40000gn/T/pip-ephem-wheel-cache-tflybflc/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
✔ Download and installation successful
Y

In [12]:
trainPath = "./semeval_train.txt"
testPath = "./semeval_test.txt"
outputTrain = "./trainSemeval.txt"
outputTest = "./testSemeval.txt"

In [22]:
def createFile(inputFilePath,outputFilePath):
    data = []
    f = open(inputFilePath, "r")
    fOut = open(outputFilePath, 'w')
    lines = f.readlines()
    for idx in range(0,len(lines),4):
        sentence_number = lines[idx].split("\t")[0]
        sentence = lines[idx].split("\t")[1][1:-1]
        label = lines[idx+1]
        sentence = sentence.replace("<e1>", " E1_START ").replace("</e1>", " E1_END ")
        sentence = sentence.replace("<e2>", " E2_START ").replace("</e2>", " E2_END ")
        tokens = nltk.word_tokenize(sentence)
        fOut.write(" ".join([ label, " ".join(tokens) ]))
        fOut.write("\n")
        e1 = tokens.index("E1_END") - 1
        e2 = tokens.index("E2_END") - 1
        sentence = " ".join(tokens)
        spacy_sentence = sp(sentence)
        new_sentence = []
        
        for word in spacy_sentence:
#             print(word.text,  word.lemma_)
            new_sentence.append(word.lemma_+"_"+word.pos_)
    
        new_sentence = " ".join(new_sentence)
        data.append([sentence_number, new_sentence, e1, e2, label])
        
    fOut.close()
    print(outputFilePath," Has been created")
    return data

In [23]:
train = createFile(trainPath,outputTrain)
test = createFile(testPath,outputTest)


./trainSemeval.txt  Has been created
./testSemeval.txt  Has been created


In [24]:
dfTrain = pd.DataFrame(data=train, columns=["sentence_number", "sentence", "e1", "e2", "relation"])
dfTrain.head()

,sentence_number,sentence,e1,e2,relation
0,1,the_DET system_NOUN as_SCONJ describe_VERB abo...,13,18,"Component-Whole(e2,e1)\n"
1,2,the_DET E1_START_PROPN child_NOUN E1_END_NOUN ...,2,12,Other\n
2,3,the_DET E1_START_PROPN author_NOUN E1_END_NOUN...,2,10,"Instrument-Agency(e2,e1)\n"
3,4,a_DET misty_ADJ E1_START_PROPN ridge_NOUN E1_E...,3,9,Other\n
4,5,the_DET E1_START_PROPN student_NOUN E1_END_PRO...,2,5,"Member-Collection(e1,e2)\n"
